<a href="https://colab.research.google.com/github/rassu-123/Abinbev-hackathon_recommender-system/blob/main/R_Abinbev_re__recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the requried libraries
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

retail_data = pd.read_excel('/content/drive/MyDrive/ABInBev GCC India __ ALe-GO_Rhythm __ Cross sell & Up sell_GAC/wholeseller_cluster.xlsx') # This may take a couple minutes

#### retail data original dataframe

In [ ]:
df_n=retail_data

In [ ]:
df_n.head()

,Unnamed: 0,Cluster,Cluster_product,Material,Ship-to nu,Brand,Postal Code,Container Size,Order qty,MACO/HL
0,0,0,0,10949,29606876,LEFFE,59160,"6,000 L",7.0,155.689825
1,3,0,5,66992,29606876,TRIPLE KARMELIET,59160,"6,000 L",2.0,200.334035
2,7,0,5,66992,29378797,TRIPLE KARMELIET,62570,"6,000 L",70.0,200.334035
3,8,0,0,10949,29378797,LEFFE,62570,"6,000 L",142.0,155.689825
4,13,5,2,70604,29379866,GOOSE,70110,"6,000 L",70.0,191.707898


In [ ]:
df_n.rename(columns={"Material":"StockCode","Brand":"Description","Ship-to nu":"CustomerID","Order qty":"Quantity"},inplace=True)

In [ ]:
df_n.head()

,Unnamed: 0,Cluster,Cluster_product,StockCode,CustomerID,Description,Postal Code,Container Size,Quantity,MACO/HL
0,0,0,0,10949,29606876,LEFFE,59160,"6,000 L",7.0,155.689825
1,3,0,5,66992,29606876,TRIPLE KARMELIET,59160,"6,000 L",2.0,200.334035
2,7,0,5,66992,29378797,TRIPLE KARMELIET,62570,"6,000 L",70.0,200.334035
3,8,0,0,10949,29378797,LEFFE,62570,"6,000 L",142.0,155.689825
4,13,5,2,70604,29379866,GOOSE,70110,"6,000 L",70.0,191.707898


In [ ]:
df_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26305 entries, 0 to 26304
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       26305 non-null  int64  
 1   Cluster          26305 non-null  int64  
 2   Cluster_product  26305 non-null  int64  
 3   StockCode        26305 non-null  int64  
 4   CustomerID       26305 non-null  int64  
 5   Description      26305 non-null  object 
 6   Postal Code      26305 non-null  int64  
 7   Container Size   26305 non-null  object 
 8   Quantity         26305 non-null  float64
 9   MACO/HL          26305 non-null  float64
dtypes: float64(2), int64(6), object(2)
memory usage: 2.0+ MB


In [ ]:
df1=df_n[df_n['CustomerID'].notna()]

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26305 entries, 0 to 26304
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       26305 non-null  int64  
 1   Cluster          26305 non-null  int64  
 2   Cluster_product  26305 non-null  int64  
 3   StockCode        26305 non-null  int64  
 4   CustomerID       26305 non-null  int64  
 5   Description      26305 non-null  object 
 6   Postal Code      26305 non-null  int64  
 7   Container Size   26305 non-null  object 
 8   Quantity         26305 non-null  float64
 9   MACO/HL          26305 non-null  float64
dtypes: float64(2), int64(6), object(2)
memory usage: 2.2+ MB


In [ ]:
len(pd.unique(df1['StockCode']))

25

In [ ]:
df2=df1[['StockCode']].drop_duplicates()

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 1713
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   StockCode  25 non-null     int64
dtypes: int64(1)
memory usage: 400.0 bytes


In [ ]:
df3=df1[['Description']].drop_duplicates()
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 630
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Description  7 non-null      object
dtypes: object(1)
memory usage: 112.0+ bytes


### df4 contains pair of unique 'StockCode' and 'Description'

In [ ]:
df4=df1[['StockCode', 'Description']].drop_duplicates() # Only get unique item/description pairs
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 1713
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   StockCode    25 non-null     int64 
 1   Description  25 non-null     object
dtypes: int64(1), object(1)
memory usage: 600.0+ bytes


#### df hi lo

In [ ]:
df5

,StockCode,Description
0,10949,LEFFE
1,66992,TRIPLE KARMELIET
4,70604,GOOSE
5,10950,HOEGAARDEN
6,33338,HOEGAARDEN
8,20617,LEFFE
9,59877,TRIPLE KARMELIET
10,71319,BUD
13,61632,KWAK
14,64668,LEFFE


In [ ]:
df_n=df

In [ ]:
df_n

,Cluster,Cluster_product,StockCode,CustomerID,Description,Postal Code,Container Size,Quantity,MACO/HL
0,0,0,10949,29606876,LEFFE,59160,"6,000 L",7.0,155.689825
1,0,0,10949,29378797,LEFFE,62570,"6,000 L",142.0,155.689825
2,0,0,10949,29379866,LEFFE,70110,"6,000 L",701.0,155.689825
3,0,0,64668,29379868,LEFFE,83630,"6,000 L",70.0,213.898843
4,0,0,10949,29379648,LEFFE,59270,"6,000 L",142.0,155.689825
...,...,...,...,...,...,...,...,...,...
4581,0,0,10949,29446268,LEFFE,53200,"6,000 L",4.0,155.689825
4582,0,0,10614,29446268,LEFFE,53200,"6,000 L",4.0,152.379409
4583,0,0,10949,29446268,LEFFE,53200,"6,000 L",1050.0,155.689825
4584,0,0,10949,29804048,LEFFE,53200,"6,000 L",5.0,155.689825


In [ ]:
df=df5

In [ ]:
df4['StockCode'] = df_n.StockCode.astype(str) # Encode as strings for future lookup ease

In [ ]:
df4

,StockCode,Description
0,10949,LEFFE
1,66992,TRIPLE KARMELIET
4,70604,GOOSE
5,10950,HOEGAARDEN
6,33338,HOEGAARDEN
8,20617,LEFFE
9,59877,TRIPLE KARMELIET
10,71319,BUD
13,61632,KWAK
14,64668,LEFFE


In [ ]:
df1['CustomerID']=df1.CustomerID.astype(int)

In [ ]:
df

,Cluster,Cluster_product,StockCode,CustomerID,Description,Postal Code,Container Size,Quantity,MACO/HL
0,0,0,10949,29606876,LEFFE,59160,"6,000 L",7.0,155.689825
1,0,0,10949,29378797,LEFFE,62570,"6,000 L",142.0,155.689825
2,0,0,10949,29379866,LEFFE,70110,"6,000 L",701.0,155.689825
3,0,0,64668,29379868,LEFFE,83630,"6,000 L",70.0,213.898843
4,0,0,10949,29379648,LEFFE,59270,"6,000 L",142.0,155.689825
...,...,...,...,...,...,...,...,...,...
4581,0,0,10949,29446268,LEFFE,53200,"6,000 L",4.0,155.689825
4582,0,0,10614,29446268,LEFFE,53200,"6,000 L",4.0,152.379409
4583,0,0,10949,29446268,LEFFE,53200,"6,000 L",1050.0,155.689825
4584,0,0,10949,29804048,LEFFE,53200,"6,000 L",5.0,155.689825


In [ ]:
df1 = df1[['StockCode', 'Quantity', 'CustomerID']] # Get rid of unnecessary info
df1

,StockCode,Quantity,CustomerID
0,10949,7.0,29606876
1,66992,2.0,29606876
2,66992,70.0,29378797
3,10949,142.0,29378797
4,70604,70.0,29379866
...,...,...,...
26300,20617,3.0,29446268
26301,17031,3.0,29446268
26302,33338,4.0,29446268
26303,53569,3.0,29446268


In [ ]:
grouped_cleaned = df1.groupby(['CustomerID', 'StockCode']).sum().reset_index() # Group together
grouped_cleaned

,CustomerID,StockCode,Quantity
0,29378797,3340,300.0
1,29378797,10949,1491.0
2,29378797,10950,82.0
3,29378797,10960,74.0
4,29378797,17031,66.0
...,...,...,...
2642,40017003,66992,14639.0
2643,40017003,70604,213.0
2644,40017003,71319,142.0
2645,40017003,77592,14.0


In [ ]:
grouped_cleaned.describe()

,CustomerID,StockCode,Quantity
count,2.647000e+03,2647.000000,2647.000000
mean,2.962797e+07,43119.698149,639.559218
std,1.372462e+06,25980.045945,2195.444064
min,2.937880e+07,3340.000000,1.000000
25%,2.937964e+07,10960.000000,55.000000
50%,2.937983e+07,53569.000000,152.000000
75%,2.944627e+07,64668.000000,422.000000
max,4.001700e+07,78359.000000,64903.660000


In [ ]:
grouped_cleaned.Quantity.loc[grouped_cleaned.Quantity == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased # This would have come as zero as the returns are recorded as negatives

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
grouped_purchased = grouped_cleaned.query('Quantity > 0') # Only get customers where purchase totals were positive

In [ ]:
grouped_purchased.describe()

,CustomerID,StockCode,Quantity
count,2.647000e+03,2647.000000,2647.000000
mean,2.962797e+07,43119.698149,639.559218
std,1.372462e+06,25980.045945,2195.444064
min,2.937880e+07,3340.000000,1.000000
25%,2.937964e+07,10960.000000,55.000000
50%,2.937983e+07,53569.000000,152.000000
75%,2.944627e+07,64668.000000,422.000000
max,4.001700e+07,78359.000000,64903.660000


In [ ]:
# Final dataset
grouped_purchased.head()

,CustomerID,StockCode,Quantity
0,29378797,3340,300.0
1,29378797,10949,1491.0
2,29378797,10950,82.0
3,29378797,10960,74.0
4,29378797,17031,66.0


In [ ]:
grouped_purchased.isna().sum()

CustomerID    0
StockCode     0
Quantity      0
dtype: int64

In [ ]:
# Creating the sparse matrix

customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased.StockCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases


In [ ]:
len(customers)

178

In [ ]:
len(products)

25

In [ ]:
# Get the associated row indices
rows = grouped_purchased.CustomerID.astype('category', pd.api.types.CategoricalDtype(categories = customers)).cat.codes 


# Get the associated column indices
cols = grouped_purchased.StockCode.astype('category', pd.api.types.CategoricalDtype(categories = products)).cat.codes 


In [ ]:
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [ ]:
purchases_sparse.shape

(178, 25)

In [ ]:
# Final sparse matrix
purchases_sparse

<178x25 sparse matrix of type '<class 'numpy.float64'>'
	with 2647 stored elements in Compressed Sparse Row format>

In [ ]:
purchases_sparse.shape[0]

178

In [ ]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
print(matrix_size)
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
print(num_purchases)
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

4450
2647


40.516853932584276

In [ ]:
import random
def make_train(ratings, pct_test = 0.2):
    test_set = ratings.copy() # Make a copy of the original set to be the test set.
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix

    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 

    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    
    random.seed(0) # Set the random seed to zero for reproducibility
    
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered
  
  # '''
  #   This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
  #   user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
  #   while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
  #   parameters: 
    
  #   ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
  #   copy of the original set. This is in the form of a sparse csr_matrix. 
    
  #   pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
  #   training set for later comparison to the test set, which contains all of the original ratings. 
    
  #   returns:
    
  #   training_set - The altered version of the original data with a certain percentage of the user-item pairs 
  #   that originally had interaction set back to zero.
    
  #   test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
  #   compares with the actual interactions.
    
  #   user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
  #   This will be necessary later when evaluating the performance via AUC.
  #   '''
    

In [ ]:
# 20% of the data has been masked for this exercise
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.1)

### Recommendation System

In [ ]:
pip install implicit

     |████████████████████████████████| 18.5 MB 411 kB/s 


In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 52.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f049a87768bd891c2ff37d412b2632c81d07a79498a8ff0ae4edc3f2356afc72
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
def nonzeros(m, row):
    """ 
    Returns the non zeroes of a row in csr_matrix
    """
    for index in range(m.indptr[row], m.indptr[row+1]):
        yield m.indices[index], m.data[index]

In [ ]:
# from numpy.ma.core import nonzero
def alternating_least_squares(Cui, factors, regularization, iterations=20):
    users, items = Cui.shape

    X = np.random.rand(users, factors) * 0.01
    Y = np.random.rand(items, factors) * 0.01

    Ciu = Cui.T.tocsr()
    for iteration in range(iterations):
        least_squares(Cui, X, Y, regularization)
        least_squares(Ciu, Y, X, regularization)

    return X, Y

def least_squares(Cui, X, Y, regularization):
    users, factors = X.shape
    YtY = Y.T.dot(Y)

    for u in range(users):
        # accumulate YtCuY + regularization * I in A
        A = YtY + regularization * np.eye(factors)

        # accumulate YtCuPu in b
        b = np.zeros(factors)

        for i, confidence in nonzeros(Cui, u):
            factor = Y[i]
            A += (confidence - 1) * np.outer(factor, factor)
            b += confidence * factor

        # Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
        X[u] = np.linalg.solve(A, b)

In [ ]:
# import implicit

alpha = 15
user_vecs, item_vecs = alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 10)

In [ ]:
type(product_train)

scipy.sparse.csr.csr_matrix

In [ ]:
user_vecs

array([[ 3.08771482,  2.20466232, 10.83383833, ...,  8.68098792,
         2.0138561 , -1.44556942],
       [ 5.78904441, -1.06747515, 11.79015054, ...,  9.49694811,
        -0.01859685,  1.49743795],
       [ 5.55044949,  3.36350917,  9.27456176, ...,  7.76746811,
         3.26335584,  4.11930351],
       ...,
       [ 2.69799337, -1.95737702,  2.80023099, ...,  5.22433233,
         3.70103384, -2.54275949],
       [ 4.41819695, -1.16162381,  5.51716281, ...,  5.56490323,
         3.32493569, -0.65127453],
       [ 3.6047834 ,  2.94379403, 10.56185203, ...,  4.6153286 ,
         2.23252342,  2.57344175]])

In [ ]:
item_vecs

### Evaluating the recommendation system using AUC-ROC curve

In [ ]:
from sklearn import metrics

def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr) 

In [ ]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark


In [ ]:
product_train

<178x25 sparse matrix of type '<class 'numpy.float64'>'
	with 2382 stored elements in Compressed Sparse Row format>

In [ ]:
len(product_users_altered)

129

In [ ]:
calc_mean_auc(product_train, product_users_altered,[sparse.csr_matrix(user_vecs),sparse.csr_matrix(item_vecs.T)],product_test)
# AUC for our recommender system

(0.827, 0.881)

### Example for recommendation

In [ ]:
len(customers)

178

In [ ]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [ ]:
len(customers_arr)

178

In [ ]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    print(prod_codes)
    df_res=pd.DataFrame()
    df_res['StockCode']=prod_codes
    # for i in range(len(prod_codes)):
    #     df_res.loc[i,1]=item_lookup.iloc[i,1]
    #df_res['Description']=item_lookup.
    return df_res
    # print(item_lookup.loc[item_lookup.StockCode.isin(prod_codes)])
    # return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [ ]:
get_items_purchased(40007894, product_train, customers_arr, products_arr, df4)

[ 3340 10949 10950 10960 17031 20617 33338 59876 59877 64668 77592 10614
 77316 63448 20530]


,StockCode
0,3340
1,10949
2,10950
3,10960
4,17031
5,20617
6,33338
7,59876
8,59877
9,64668


In [ ]:
get_items_purchased(40007894, product_train, customers_arr, products_arr, df4)

[ 3340 10949 10950 10960 17031 20617 33338 59876 59877 64668 77592 10614
 77316 63448 20530]


,StockCode
0,3340
1,10949
2,10950
3,10960
4,17031
5,20617
6,33338
7,59876
8,59877
9,64668


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    #print(cust_ind)
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    #print(pref_vec)
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    #print(pref_vec)
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) 
    #print(rec_vector)
    # print(rec_vector)
    # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    # rec_vector.reshape(-1,1)
    # min_max = scale_sparse_matrix_rows(rec_vector)

    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    #print(rec_vector_scaled)
   # rec_vector_scaled=min_max[:,0]
   # print(len(pref_vec))
   # print(len(rec_vector_scaled))
    recommend_vector = pref_vec*rec_vector_scaled 
    # print(recommend_vector)
    # print("h")
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    print(product_idx)
    # of best recommendations
    rec_list = [] # start empty list to store items
    df_res2=pd.DataFrame()
    
    # for i in range(len(prduct_idx)):
    #     df_res2.loc[i,1]=item_lookup.iloc[i,1]
    #df_res['Description']=item_lookup.
    # print(df_res)
    for index in product_idx:
        code = item_list[index]
        #print(code)
        rec_list.append(code)
    df_res2['StockCode']=rec_list

    return df_res2
    
        

        # print(code)

        #rec_list.append([code, item_lookup.Description.loc[item_lookup.StockCode == code].iloc[0]]) 
        # Append our descriptions to the list
    # for j in range(len(df_res2)):
    #      df_res2.loc[j,1]=item_lookup.iloc[j,1]
    # print(df_res2)
    # codes = [item[0] for item in rec_list]
    # print(codes)
    # descriptions = [item[1] for item in rec_list]
    # final_frame = pd.DataFrame({'StockCode': codes, 'Description': descriptions}) # Create a dataframe 
    # return final_frame[['StockCode', 'Description']] # Switch order of columns around


In [ ]:
rec_items(40007894, product_train, user_vecs, item_vecs, customers_arr, products_arr, df4,
                       num_items = 5)

[ 9 15 12 13 14]


,StockCode
0,53569
1,71319
2,61632
3,66992
4,70604


In [ ]:
# Let's try another example
get_items_purchased(12353, product_train, customers_arr, products_arr, df4)

,StockCode,Description
3594,22559,SEASIDE FLYING DISC
3637,21158,MOODY GIRL DOOR HANGER
231161,23125,6PC WOOD PLATE SET DISPOSABLE
325437,23411,TRELLIS COAT RACK


In [ ]:
rec_items(12353, product_train, user_vecs, item_vecs.T, customers_arr, products_arr, df4,
                       num_items = 10)

,StockCode,Description
0,23166,MEDIUM CERAMIC TOP STORAGE JAR
